In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import cpca
import os
import math
os.chdir("/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data/抗议数据")


In [84]:
df = pd.read_csv("/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data/抗议数据/RFA_protest3.csv",encoding='utf-8')
df = df[['adcode','location','size_level','year','month','day','citycode','省','市','区']]
df.to_csv("/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data/抗议数据/RFA_protest3_cropped.csv",index=False)

In [2]:
df = pd.read_stata("cloudseeding.dta")
df["district"] = df['prov'] + df['city'] + df['county']
df_adcode = cpca.transform(df["district"])
df['adcode'] = df_adcode['adcode']
df.to_csv("cloudseeding_adcode.csv")

In [12]:
df = pd.read_stata("Meteorological.dta")
df["district"] = df['prov'] + df['city'] + df['county']
df_adcode = cpca.transform(df["district"])
df['adcode'] = df_adcode['adcode']
df.to_csv("meteorological_adcode.csv")

In [24]:
df = pd.read_stata("final_panel_newweibo.dta")
df = df.sort_values(by=['adcode', 'date']).reset_index()
df["day"] = df.groupby("adcode").cumcount()
df['event'] = 0

# Identify events (first protest and subsequent protests >= 3 months apart)
for ad, ad_df in df.groupby('adcode'):
    protest_dates = ad_df.loc[ad_df['n_prt_weibo'] > 0, 'day'].sort_values().tolist()
    last_event = None
    
    for protest_date in protest_dates:
        if last_event is None or (protest_date - last_event) >= 45:
            df.loc[(df['adcode'] == int(ad)) & (df['day'] == protest_date),'event'] = 1
            last_event = protest_date

index_list = df.loc[df['event']==1,'index'].tolist()
df['to_day']=None
for index in index_list:
    for i in range(-22,23):
        if not (index+i<0) or (index+i>len(df)):
            df.loc[df['index']==index+i,'to_day'] = i

df.to_csv('eventstudy_weibo_county.csv')

In [25]:
df = pd.read_stata("final_panel_newweibo.dta")
df = df.sort_values(by=['adcode', 'date']).reset_index()
df["day"] = df.groupby("adcode").cumcount()
df['event'] = 0

# Identify events (first protest and subsequent protests >= 3 months apart)
for ad, ad_df in df.groupby('adcode'):
    protest_dates = ad_df.loc[ad_df['n_prt_rfa'] > 0, 'day'].sort_values().tolist()
    last_event = None
    
    for protest_date in protest_dates:
        if last_event is None or (protest_date - last_event) >= 45:
            df.loc[(df['adcode'] == int(ad)) & (df['day'] == protest_date),'event'] = 1
            last_event = protest_date

index_list = df.loc[df['event']==1,'index'].tolist()
df['to_day']=None
for index in index_list:
    for i in range(-22,23):
        if not (index+i<0) or (index+i>len(df)):
            df.loc[df['index']==index+i,'to_day'] = i

df.to_csv('eventstudy_rfa_county.csv')

In [6]:
df = pd.read_stata("eventstudy_city.dta")
df = df.sort_values(by=['citycode', 'date']).reset_index()
df["day"] = df.groupby("citycode").cumcount()
df['event'] = 0

# Identify events (first protest and subsequent protests >= 3 months apart)
for city, city_df in df.groupby('citycode'):
    protest_dates = city_df.loc[city_df['n_prt_weibo'] > 0, 'day'].sort_values().tolist()
    last_event = None
    
    for protest_date in protest_dates:
        if last_event is None or (protest_date - last_event) >= 45:
            df.loc[(df['citycode'] == int(city)) & (df['day'] == protest_date),'event'] = 1
            last_event = protest_date

index_list = df.loc[df['event']==1,'index'].tolist()
df['to_day']=None
for index in index_list:
    for i in range(-22,23):
        if not (index+i<0) or (index+i>len(df)):
            df.loc[df['index']==index+i,'to_day'] = i

df.to_csv('eventstudy_weibo_city.csv')

In [7]:
df = pd.read_stata("eventstudy_city.dta")
df = df.sort_values(by=['citycode', 'date']).reset_index()
df["day"] = df.groupby("citycode").cumcount()
df['event'] = 0

# Identify events (first protest and subsequent protests >= 3 months apart)
for city, city_df in df.groupby('citycode'):
    protest_dates = city_df.loc[city_df['n_prt_rfa'] > 0, 'day'].sort_values().tolist()
    last_event = None
    
    for protest_date in protest_dates:
        if last_event is None or (protest_date - last_event) >= 45:
            df.loc[(df['citycode'] == int(city)) & (df['day'] == protest_date),'event'] = 1
            last_event = protest_date

index_list = df.loc[df['event']==1,'index'].tolist()
df['to_day']=None
for index in index_list:
    for i in range(-22,23):
        if not (index+i<0) or (index+i>len(df)):
            df.loc[df['index']==index+i,'to_day'] = i

df.to_csv('eventstudy_rfa_city.csv')

In [ ]:
df = pd.read_csv('eventstudy_weibo_city.csv')

# Identify unique events and assign an event ID 
df_list = []
for city, city_df in df.groupby('citycode'):
    city_df['event_id'] = (city_df['event'] == 1).cumsum()
    city_df.loc[df['event'] == 0, 'event_id'] = None   
    num = city_df['event_id'].max()

    if math.isnan(num):
        city_df['num'] = 0
        df_list.append(city_df)
    else:
        for i in range(0,int(num)):
            city_df_temp = city_df.copy()
            city_df_temp['num'] = i+1
            df_list.append(city_df_temp)

df_final = pd.concat(df_list)
df_final.to_stata('temp_event.dta')

In [3]:
df = pd.read_csv('eventstudy_rfa_city.csv')

# Identify unique events and assign an event ID 
df_list = []
for city, city_df in df.groupby('citycode'):
    city_df['event_id'] = (city_df['event'] == 1).cumsum()
    city_df.loc[df['event'] == 0, 'event_id'] = None   
    num = city_df['event_id'].max()

    if math.isnan(num):
        city_df['num'] = 0
        df_list.append(city_df)
    else:
        for i in range(0,int(num)):
            city_df_temp = city_df.copy()
            city_df_temp['num'] = i+1
            df_list.append(city_df_temp)

df_final = pd.concat(df_list)
df_final.to_stata('temp_event_rfa.dta')


/var/folders/5h/rb902v091ys_0c5x0jt8sbjw0000gp/T/ipykernel_57071/1475265588.py:20: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Unnamed: 0   ->   Unnamed__0

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  df_final.to_stata('temp_event_rfa.dta')


In [22]:
import ollama
import pandas as pd
import csv
import os
from multiprocessing import Pool

folder_dir  = r"/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data/抗议数据"
input_file   = rf"{folder_dir}/extracted_weibo_time2.csv"
output_file = rf"{folder_dir}/extracted_weibo_time3.csv"

df = pd.read_csv(input_file, encoding="utf-8",dtype={"省":"string","市":"string","区":"string"},index_col=False )

df['time2'] = df['time2'].astype(str)
df['time2'] = df['time2'].str.replace("[", "")
df['time2'] = df['time2'].str.replace("]", "")
df['time2'] = df['time2'].str.replace("，", ",")
df['time2'] = df['time2'].str.replace("、", ",")
df['time2'] = df['time2'].str.replace("；", ",")
df['time2'] = df['time2'].str.replace(";", ",")
df['time2'] = df['time2'].str.split(',')

df = df.explode('time2', ignore_index=True)
df[['month1','year1','day1']] = None
df[['month1','year1','day1']] = df[['month1','year1','day1']].astype(str)
df['month1'] = df['time2'].str.extract(r'(\d+)(?=月)')
df['year1'] = df['time2'].str.extract(r'(\d+)(?=年)')
df['day1'] = df['time2'].str.extract(r'(\d+)(?=日)')
df.loc[df['day1'].isna(), 'day1'] = df['time2'].str.extract(r'(\d+)(?=号)')

df1= df.loc[(df['year1'].isna()) | (df['month1'].isna()) | (df['day1'].isna())]
df1['time1'] = df1['time1'].astype(str)
df1['time1'] = df1['time1'].str.replace("[", "")
df1['time1'] = df1['time1'].str.replace("]", "")
df1['time1'] = df1['time1'].str.replace("，", ",")
df1['time1'] = df1['time1'].str.replace("、", ",")
df1['time1'] = df1['time1'].str.replace("；", ",")
df1['time1'] = df1['time1'].str.replace(";", ",")
df1['time1'] = df1['time1'].str.split(',')

df1 = df1.explode('time1', ignore_index=True)
df1 = df1[df1['time1'].str.contains(r'[号日]', na=False)]

df1.loc[df1['month1'].isna(), 'month1'] = df1['time1'].str.extract(r'(\d+)(?=月)')[0]
df1.loc[df1['year1'].isna(), 'year1'] = df1['time1'].str.extract(r'(\d+)(?=年)')[0]
df1.loc[df1['day1'].isna(), 'day1'] = df1['time1'].str.extract(r'(\d+)(?=日)')[0]
df1.loc[df1['day1'].isna(), 'day1'] = df1['time1'].str.extract(r'(\d+)(?=号)')[0]

# replace year1 = year if year1.isna() or if year1<2010
df1['year1'] = df1['year1'].astype('Int64')
df1['month1'] = df1['month1'].astype('Int64')
df1.loc[df1['year1'].isna() | (df1['year1'] < 2010), 'year1'] = df1['year']
df1.loc[df1['month1'].isna(), 'month1'] = df1['month']

df2 = pd.concat([df,df1])
df2.dropna(subset=['year1', 'month1','day1'],inplace=True)
df2.drop_duplicates(subset=['posts','year1', 'month1','day1'],inplace=True)
df2.to_csv('cleaned_time_weibo_protests.csv')

/var/folders/5h/rb902v091ys_0c5x0jt8sbjw0000gp/T/ipykernel_11317/258707157.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['time1'] = df1['time1'].astype(str)
/var/folders/5h/rb902v091ys_0c5x0jt8sbjw0000gp/T/ipykernel_11317/258707157.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['time1'] = df1['time1'].str.replace("[", "")
/var/folders/5h/rb902v091ys_0c5x0jt8sbjw0000gp/T/ipykernel_11317/258707157.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data